In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [2]:
from utils.functions import *

In [3]:
data = TimeSeriesDataFrame.from_path(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/grocery_sales/test.csv",
)
data.head()

scaled_price  promotion_email  promotion_homepage  \
item_id  timestamp                                                       
1062_101 2018-01-01      0.879130              0.0                 0.0   
         2018-01-08      0.994517              0.0                 0.0   
         2018-01-15      1.005513              0.0                 0.0   
         2018-01-22      1.000000              0.0                 0.0   
         2018-01-29      0.883309              0.0                 0.0   

                     unit_sales  
item_id  timestamp               
1062_101 2018-01-01       636.0  
         2018-01-08       123.0  
         2018-01-15       391.0  
         2018-01-22       339.0  
         2018-01-29       661.0

In [5]:
prediction_length = 8
train_data, test_data = data.train_test_split(prediction_length=prediction_length)

In [6]:
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="unit_sales",
    known_covariates_names=["scaled_price", "promotion_email", "promotion_homepage"],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            # Zero-shot model WITHOUT covariates
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"},
            },
            # Chronos-Bolt (Small) combined with CatBoost on covariates
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=60,
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/home/antares/Documents/PoliTo/Fall_24/DeepNLP/DeepNLP_Forecasting/AutogluonModels/ag-20250517_144135'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202412060638~1743109366~22.04~1fce33b SMP PREEMPT_DYNAMIC Thu M
CPU Count:          16
GPU Count:          1
Memory Avail:       3.70 GB / 15.32 GB (24.1%)
Disk Space Avail:   320.51 GB / 441.91 GB (72.5%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'WithRegressor'},
                                  'covariate_regressor': 'CAT',
                                  'model_path': 'bolt_small',
                   

In [7]:
predictor.leaderboard(test_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,ChronosWithRegressor[bolt_small],-0.268969,-0.358048,0.978846,0.872210,0.437420,2
1,ChronosZeroShot[bolt_small],-0.318562,-0.452296,0.956743,1.556524,2.735768,1


In [8]:
predictions3 = predictor.predict(
    test_data.iloc[: -prediction_length], known_covariates=test_data.iloc[-prediction_length:])

ValueError: known_covariates are missing information for the following item_ids: ['1062_101', '1062_102', '1062_104', '1062_106', '1062_108', '1062_109', ...].

In [11]:
test_data.loc[:,["scaled_price", "promotion_email", "promotion_homepage"]]

scaled_price  promotion_email  promotion_homepage
item_id  timestamp                                                    
1062_101 2018-01-01      0.879130              0.0                 0.0
         2018-01-08      0.994517              0.0                 0.0
         2018-01-15      1.005513              0.0                 0.0
         2018-01-22      1.000000              0.0                 0.0
         2018-01-29      0.883309              0.0                 0.0
...                           ...              ...                 ...
2956_108 2018-07-02      1.001721              0.0                 0.0
         2018-07-09      0.998282              0.0                 0.0
         2018-07-16      0.996570              0.0                 0.0
         2018-07-23      1.000000              0.0                 0.0
         2018-07-30      1.001718              0.0                 0.0

[9889 rows x 3 columns]